In [8]:
import pandas as pd
import numpy as np
import tqdm
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.stattools import adfuller
from scipy.stats import boxcox
from arch import arch_model

from warnings import filterwarnings 
filterwarnings('ignore')

In [2]:
sales_of_company = pd.read_csv('monthly-sales-of-company-x-jan-6.csv')
robberies_in_boston = pd.read_csv('monthly-boston-armed-robberies-j.csv')
#airlines_passengers = pd.read_csv("international-airline-passengers.csv")
mean_monthly_temp = pd.read_csv("mean-monthly-air-temperature-deg.csv")
dowjones_closing = pd.read_csv("weekly-closings-of-the-dowjones-.csv")
female_births = pd.read_csv("daily-total-female-births-in-cal.csv")

all_series = {
    "Sales": sales_of_company["Count"],
    "Robberies": robberies_in_boston["Count"],
    "Temperature": mean_monthly_temp["Deg"],
    "Dow_Jones": dowjones_closing["Close"],
    "Births": female_births["Count"]
}

In [3]:
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

Построим ARCH и GARCH модели для разных рядов

Для GARCH моделей будем использовать "лучшие" параметры ARIMA модели

1. Ряд Sales

In [6]:
test_stationarity(all_series['Sales'])

Results of Dickey-Fuller Test:
Test Statistic                  0.654715
p-value                         0.988889
#Lags Used                     12.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64


тест Дики-Фуллера не позволяет отвергнуть гипотезу о нестационарности ряда. Попробуем обработать ряд при помощи преобразования Бокса-Кокса и дифференцирования

In [9]:
sales_new = boxcox(all_series['Sales'], 0)
sales_new_1 = np.diff(sales_new, 1)

In [10]:
test_stationarity(sales_new_1)

Results of Dickey-Fuller Test:
Test Statistic                 -3.135644
p-value                         0.024025
#Lags Used                     11.000000
Number of Observations Used    64.000000
Critical Value (1%)            -3.536928
Critical Value (5%)            -2.907887
Critical Value (10%)           -2.591493
dtype: float64


Теперь тест Дики-Фуллера отвергает гипотезу о нестационарности ряда

Используем обработанные данные для построения ARCH и GARCH моделей

In [11]:
%%time

best_aic_sales = np.inf 
best_order_sales = None
best_mdl_sales = None

for i in range(10):
    for d in range(10):
        for j in range(10):
            try:
                tmp_mdl = smt.ARIMA(sales_new_1, order=(i,d,j)).fit(method='mle', trend='nc')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic_sales:
                    best_aic_sales = tmp_aic
                    best_order_sales = (i, d, j)
                    best_mdl_sales = tmp_mdl
            except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic_sales, best_order_sales))


aic: -37.58410 | order: (7, 0, 2)
Wall time: 1min 31s


In [12]:
p_sales, o_sales, q_sales = best_order_sales

am_sales_garch = arch_model(sales_new_1, p=p_sales, o=o_sales, q=q_sales, dist='StudentsT')
res_sales_garch = am_sales_garch.fit(update_freq=5, disp='off')

am_sales_arch = arch_model(sales_new_1, p=p_sales, o=o_sales, q=q_sales, dist='StudentsT', vol='ARCH')
res_sales_arch = am_sales_arch.fit(update_freq=5, disp='off')
print('GARCH model')
print(res_sales_garch.summary())
print('+++++++++++++++++++++++++')
print('ARCH model')
print(res_sales_arch.summary())

GARCH model
                        Constant Mean - GARCH Model Results                         
Dep. Variable:                            y   R-squared:                      -0.003
Mean Model:                   Constant Mean   Adj. R-squared:                 -0.003
Vol Model:                            GARCH   Log-Likelihood:               -16.7007
Distribution:      Standardized Student's t   AIC:                           57.4014
Method:                  Maximum Likelihood   BIC:                           85.3702
                                              No. Observations:                   76
Date:                      Wed, Feb 10 2021   Df Residuals:                       64
Time:                              16:01:01   Df Model:                           12
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-------------------------------------------------------------------

2. Ряд Dow_Jones

In [13]:
test_stationarity(all_series['Dow_Jones'])

Results of Dickey-Fuller Test:
Test Statistic                  -1.314625
p-value                          0.622455
#Lags Used                       0.000000
Number of Observations Used    161.000000
Critical Value (1%)             -3.471633
Critical Value (5%)             -2.879665
Critical Value (10%)            -2.576434
dtype: float64


тест Дики-Фуллера не позволяет отвергнуть гипотезу о нестационарности ряда. Попробуем обработать ряд при помощи преобразования Бокса-Кокса и дифференцирования

In [16]:
DJ_new = boxcox(all_series['Dow_Jones'], 0)
DJ_new_1 = np.diff(DJ_new, 1)
test_stationarity(DJ_new_1)

Results of Dickey-Fuller Test:
Test Statistic                -1.299582e+01
p-value                        2.751609e-24
#Lags Used                     0.000000e+00
Number of Observations Used    1.600000e+02
Critical Value (1%)           -3.471896e+00
Critical Value (5%)           -2.879780e+00
Critical Value (10%)          -2.576495e+00
dtype: float64


Тест Дики-Фуллера отвергает гипотезу о нестационарности ряда

Используем обработанные данные для построения ARCH и GARCH моделей

In [17]:
%%time

best_aic_dj = np.inf 
best_order_dj = None
best_mdl_dj = None

for i in range(10):
    for d in range(10):
        for j in range(10):
            try:
                tmp_mdl = smt.ARIMA(DJ_new_1, order=(i,d,j)).fit(method='mle', trend='nc')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic_dj:
                    best_aic_dj = tmp_aic
                    best_order_dj = (i, d, j)
                    best_mdl_dj = tmp_mdl
            except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic_dj, best_order_dj))

#print(best_mdl_dj.summary())
#tsplot(best_mdl_dj.resid, lags=30)

aic: -770.01581 | order: (2, 0, 2)
Wall time: 9min 3s


In [18]:
p_dj, o_dj, q_dj = best_order_dj

# Using student T distribution usually provides better fit
am_dj = arch_model(DJ_new_1, p=p_dj, o=o_dj, q=q_dj, dist='StudentsT')
res_dj = am_dj.fit(update_freq=5, disp='off')
print(res_dj.summary())

                        Constant Mean - GARCH Model Results                         
Dep. Variable:                            y   R-squared:                      -0.005
Mean Model:                   Constant Mean   Adj. R-squared:                 -0.005
Vol Model:                            GARCH   Log-Likelihood:                393.514
Distribution:      Standardized Student's t   AIC:                          -773.029
Method:                  Maximum Likelihood   BIC:                          -751.459
                                              No. Observations:                  161
Date:                      Wed, Feb 10 2021   Df Residuals:                      154
Time:                              16:25:40   Df Model:                            7
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
m

3. Ряд Births

In [19]:
test_stationarity(all_series['Births'])

Results of Dickey-Fuller Test:
Test Statistic                  -4.808291
p-value                          0.000052
#Lags Used                       6.000000
Number of Observations Used    358.000000
Critical Value (1%)             -3.448749
Critical Value (5%)             -2.869647
Critical Value (10%)            -2.571089
dtype: float64


Тест Дики-Фуллера отвергает гипотезу о нестационарности ряда

In [20]:
%%time

best_aic_b = np.inf 
best_order_b = None
best_mdl_b = None

for i in range(7):
    for d in range(7):
        for j in range(7):
            try:
                tmp_mdl = smt.ARIMA(all_series['Births'], order=(i,d,j)).fit(method='mle', trend='nc')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic_b:
                    best_aic_b = tmp_aic
                    best_order_b = (i, d, j)
                    best_mdl_b = tmp_mdl
            except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic_b, best_order_b))

#print(best_mdl_b.summary())
#tsplot(best_mdl_b.resid, lags=30)

aic: 2452.86209 | order: (3, 1, 4)
Wall time: 1min 37s


In [21]:
p_, o_, q_ = best_order_b

# Using student T distribution usually provides better fit
am = arch_model(all_series['Births'], p=p_, o=o_, q=q_, dist='StudentsT')
res = am.fit(update_freq=5, disp='off')
print(res.summary())

                      Constant Mean - GJR-GARCH Model Results                       
Dep. Variable:                        Count   R-squared:                      -0.001
Mean Model:                   Constant Mean   Adj. R-squared:                 -0.001
Vol Model:                        GJR-GARCH   Log-Likelihood:               -1241.35
Distribution:      Standardized Student's t   AIC:                           2504.70
Method:                  Maximum Likelihood   BIC:                           2547.60
                                              No. Observations:                  365
Date:                      Wed, Feb 10 2021   Df Residuals:                      354
Time:                              16:29:32   Df Model:                           11
                               Mean Model                               
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu            41